## Importing Liabraies

In [136]:
import pandas as pd
import numpy as np
import nltk

## Loading Data

In [137]:
data = pd.read_csv('spam.csv',encoding = 'latin')

In [138]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


<font color = 'red'> Dropping these unamed columns
   

In [139]:
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 1)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


<font color = 'red'> For Good unerstanding Changing Labels

In [140]:
data.rename(columns = {'v1':'Label',
                     'v2':'Messege'},
         inplace = True)

data.head()

,Label,Messege
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


<font color = 'red'> Changing the Value of 'Label' column to 0,1<br>
    
not doing manually because then it will become priority

## Handeling Categorical Data

In [141]:
data = pd.get_dummies(data,columns=['Label'])

data.head()

,Messege,Label_ham,Label_spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0


In [142]:
#Checking Ham and Spam distribution

data['Label_ham'].value_counts()

1    4825
0     747
Name: Label_ham, dtype: int64

In [143]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Messege     5572 non-null   object
 1   Label_ham   5572 non-null   uint8 
 2   Label_spam  5572 non-null   uint8 
dtypes: object(1), uint8(2)
memory usage: 54.5+ KB


#### Total length of msg in our data

In [144]:
data['count'] = 0

for i in np.arange(0,len(data['Messege'])):
    data.loc[i,'count'] = len(data.loc[i,'Messege'])
    
    
data.head()

,Messege,Label_ham,Label_spam,count
0,"Go until jurong point, crazy.. Available only ...",1,0,111
1,Ok lar... Joking wif u oni...,1,0,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1,155
3,U dun say so early hor... U c already then say...,1,0,49
4,"Nah I don't think he goes to usf, he lives aro...",1,0,61


In [145]:
data.describe()

,Label_ham,Label_spam,count
count,5572.000000,5572.000000,5572.000000
mean,0.865937,0.134063,80.118808
std,0.340751,0.340751,59.690841
min,0.000000,0.000000,2.000000
25%,1.000000,0.000000,36.000000
50%,1.000000,0.000000,61.000000
75%,1.000000,0.000000,121.000000
max,1.000000,1.000000,910.000000


<font color = 'red'> Messege column is not showing because it is having text data not numeric data

# <font color = 'green'> Processing Messege (S-2)

<font color = 'blue'> Will use NLP to Process Messege column to pass in the ML model

In [146]:
# Viewing first-2 Orignal Messeges

print(data['Messege'][0],)
print()
print(data['Messege'][1])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

Ok lar... Joking wif u oni...


In [147]:
data['Messege']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Messege, Length: 5572, dtype: object

## Preparing WordVector Corpus

In [148]:
corpus = []

## Using Porter Stemmer

In [149]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [150]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

#importing Regular Expression
import re

In [151]:
for i in range(0,5572):
    
    # Using Regular Expression
    msg = data['Messege'][i]
    
    # e-mail
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', msg)
    
    # URL
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',msg)
    
    # Money
    msg = re.sub('([A-Z]{3}|[A-Z]?[\$€¥])?\s?(\d{1,3}((,\d{1,3})+)?(.\d{1,3})?(.\d{1,3})?(,\d{1,3})?)', 'moneysymb', msg)
    
    # Phone_Number
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', msg)
    
    # Numbers
    msg = re.sub('\d+(\.\d+)?', 'numbr', msg)
    
    # Removing Punctuation Marks
    msg = re.sub('[^\w\d\s]', ' ', msg)
    
    #Printing these messeges to see how they are now
    
    if i<2:
        print("\t\t\t\t\t\t\t\t\ Messege",i)
    if i<2:
        print('\nRegular Expression',i,':',msg)
        
    
    # Lowering all these words
    msg=msg.lower()
    
    if i<2:
        print('\nAfter Lowering : ',msg)
    
    # Using Split function to find tokens
    msg=msg.split()
    
    if i<2:
        print('\nAfter Splitting : ',msg)
        
    # Seperating STOP-WORDS and applying Steming techinque
    msg = [ ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
    
    if i<2:
        print('\nAfter steming and stopword removal : ',msg)
        
        
    # JOINING the messeges
    msg = ' '.join(msg)
    
    if i<2:
        print('\nFinal Messege: ',msg)
        
    # Preparing WordVector Corpus
    corpus.append(msg)
    if i<2:
        print(corpus)

								\ Messege 0

Regular Expression 0 : Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   

After Lowering :  go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   

After Splitting :  ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']

After steming and stopword removal :  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']

Final Messege:  go jurong point crazi avail bugi n great world la e buffet cine got amor wat
['go jurong point crazi avail bugi n great world la e buffet cine got amor wat']
								\ Messege 1

Regular Expression 1 : Ok lar    Joking wif u oni   

After Lowering :  ok lar    joking wif u oni   

After Splitting :  ['ok', 'lar', 'joking', 'wif', 'u', 'oni']

Af

# <font color = 'green'> Preparing Vector For Each Image

In [152]:
cv = CountVectorizer()
cv

CountVectorizer()

In [154]:
data_input = cv.fit_transform(corpus).toarray()
data_input

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [155]:
data_input[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [156]:
data_input[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# <font color = 'green'> Applying Classification
    
<ul>
    <li> <b>Input</b> - Prepared sparse matrix/vector for each messege</li>
    <li> <b>Output</b> - label i.e. , Spam or Ham

In [157]:
data.head()

,Messege,Label_ham,Label_spam,count
0,"Go until jurong point, crazy.. Available only ...",1,0,111
1,Ok lar... Joking wif u oni...,1,0,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1,155
3,U dun say so early hor... U c already then say...,1,0,49
4,"Nah I don't think he goes to usf, he lives aro...",1,0,61


In [158]:
data_output = data['Label_ham']

## Splitting Data for Traning and Testing

In [160]:
from sklearn.model_selection import train_test_split

In [161]:
data_input.shape

(5572, 6894)

In [162]:
data_output.shape

(5572,)

In [164]:
train_x, test_x, train_y, test_y = train_test_split(data_input, data_output, test_size=0.2, random_state=0)

<font color = 'red'> Now we can initialize our ML models<br>
    
    1. GausianNB
    2. DecisionTreeClassifier
    3. RandomForestClassifier

# <font color ='green'> Initializing ML Models

In [166]:
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Preparing ML Models

### Traning

In [168]:
# naive_bayes
model_nvb = GaussianNB()
model_nvb.fit(train_x,train_y)

# DecisionTreeClassifier
model_dt = tree.DecisionTreeClassifier()
model_dt.fit(train_x,train_y)

#RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators = 100)
model_rf.fit(train_x,train_y)

RandomForestClassifier()

## Model Predictions

In [170]:
# Predictions for NaiveBayes
prediction_nvb = model_nvb.predict(test_x)

# Predictions for DesicionTreeClassifier
prediction_dt = model_dt.predict(test_x)

# Predictions for RandomForestClassifier
prediction_rf = model_rf.predict(test_x)

## Results Naive Bayes

In [172]:
# Accuracy
print("Accuracy : %0.5f \n\n" % accuracy_score(test_y,prediction_nvb))

# Classification Report
print("Classification Report : \n",classification_report(test_y,prediction_nvb))

Accuracy : 0.86637 


Classification Report : 
               precision    recall  f1-score   support

           0       0.53      0.85      0.65       166
           1       0.97      0.87      0.92       949

    accuracy                           0.87      1115
   macro avg       0.75      0.86      0.79      1115
weighted avg       0.91      0.87      0.88      1115



## Results DecisionTreeClassifier

In [173]:
# Accuracy
print("Accuracy : %0.5f \n\n" % accuracy_score(test_y,prediction_dt))

# Classification Report
print("Classification Report : \n",classification_report(test_y,prediction_dt))

Accuracy : 0.97130 


Classification Report : 
               precision    recall  f1-score   support

           0       0.94      0.86      0.90       166
           1       0.98      0.99      0.98       949

    accuracy                           0.97      1115
   macro avg       0.96      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115



## Results RandomForestClassifer

In [174]:
# Accuracy
print("Accuracy : %0.5f \n\n" % accuracy_score(test_y,prediction_rf))

# Classification Repo
print("Classification Report : \n",classification_report(test_y,prediction_rf))

Accuracy : 0.97578 


Classification Report : 
               precision    recall  f1-score   support

           0       1.00      0.84      0.91       166
           1       0.97      1.00      0.99       949

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.97      1115

